In [ ]:
from astropy.time import Time, TimeDelta
import numpy as np

from lsst_efd_client import EfdClient

In [ ]:
#efd_name = "ldf_stable_efd"
#efd_name = "base_efd"
efd_name = "ncsa_teststand_efd"
check_date = "2021-03-10"

In [ ]:
client = EfdClient(efd_name)

In [ ]:
start_time = Time(check_date, scale="tai")
end_time = TimeDelta(24*3600, format="sec", scale="tai")
si_df = await client.select_time_series("lsst.sal.CCCamera.logevent_startIntegration",
                                        ["private_sndStamp", "additionalValues"],
                                        start=start_time, end=end_time)
eoit_df = await client.select_time_series("lsst.sal.CCCamera.logevent_endOfImageTelemetry",
                                          ["private_sndStamp", "additionalValues"],
                                          start=start_time, end=end_time)

In [ ]:
grp_si = si_df.groupby("additionalValues")
grp_eoit = eoit_df.groupby("additionalValues")

In [ ]:
for i in grp_si.groups.keys():
    try:
        print(i)
        img_si = grp_si.get_group(i)
        img_eoit = grp_eoit.get_group(i)
        si_timestamps = img_si["private_sndStamp"].values
        eoit_timestamps = img_eoit["private_sndStamp"].values
        diff_time = eoit_timestamps[:-1] - si_timestamps[1:]
        elapsed_time = eoit_timestamps - si_timestamps
        print(f"Num Images: {si_timestamps.size}")
        neg_diffs = diff_time < 0
        unique, counts = np.unique(neg_diffs, return_counts=True)
        diff_counts = dict(zip(unique, counts))
        try:
            print(f"endOfImageTelemetry BEFORE next startIntegration: {diff_counts[True]}")
        except KeyError:
            pass
        try:
            print(f"endOfImageTelemetry AFTER next startIntegration: {diff_counts[False]}")
        except KeyError:
            pass
        print(f"Mean Elapsed Time (sec): {np.mean(elapsed_time):.3f}")
    except KeyError:
        print("Badly formatted key.")